In [1]:
from pcdsdevices.mirror import OffsetMirror
from pcdsdevices.mirror import PointingMirror
import datetime
from functions import *
from timeout import timeout

import logging
# create logger
level = logging.INFO
format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
handlers = [logging.FileHandler('HOMS_test.log'), logging.StreamHandler()]
logging.basicConfig(level = level, format = format, handlers = handlers)
logger = logging.getLogger(__name__)
logger.info("\nTest initiated at: %s\n" %datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

2018-07-12 16:16:01,377 - __main__ - INFO - 
Test initiated at: 2018-07-12 16:16



In [2]:
if get_user_confirmation() == False:
    import sys; sys.exit("aa! exit!!")

This script moves real hardware.
Make sure limit switches are working.
Do you want to continue(Y/N)?y


In [3]:
fee_m1 = OffsetMirror('FEE1:M1H', name='fee_m1', prefix_xy='STEP:M1H', xgantry_prefix='STEP:FEE1:611:MOTR')
fee_m2 = OffsetMirror('FEE1:M2H', name='fee_m2', prefix_xy='STEP:M2H', xgantry_prefix='STEP:FEE1:861:MOTR')
xrt_m1 = PointingMirror('XRT:M1H', name='xrt_m1')
xrt_m2 = PointingMirror('XRT:M2H', name='xrt_m2')
xrt_m3 = OffsetMirror('XRT:M3H', name='xrt_m3')

In [4]:
while True:
    try:
        which_mirror = int(input("which mirror do you want to use?\n press:\n 1 -> fee_m1; 2 -> fee_m2; 3 -> xrt_m1; 4 -> xrt_m2; 5  -> xrt_m3."))
    except ValueError:
        print ("sorry, I didn't understand that")
        continue
    if not which_mirror in range (1,6):
        print("sorry, it's not one of possible inputs")
        continue
    else:
        break
        
which_mirror_dict = {1: fee_m1, 2:fee_m2, 3:xrt_m1, 4:xrt_m2, 5:xrt_m3}
test_object = which_mirror_dict[which_mirror]
logger.info("\tusing %s mirror" %test_object.name)

which mirror do you want to use?
 press:
 1 -> fee_m1; 2 -> fee_m2; 3 -> xrt_m1; 4 -> xrt_m2; 5  -> xrt_m3.3


2018-07-12 16:16:18,443 - __main__ - INFO - 	using xrt_m1 mirror


In [5]:
# FOR ALL AXES
logger.info("\t%.4f recording original position...\n" %test_object.xgantry.readback.timestamp)
initial_mirror_angle = test_object.pitch()
mirror_angle_unit = test_object.pitch.egu
initial_xgantry_value = test_object.xgantry()
initial_ygantry_value = test_object.ygantry()
gantry_value_unit = test_object.xgantry.egu
logger.info("initial mirror angle is %.4f %s" %(initial_mirror_angle, test_object.pitch.egu))
logger.info("initial xgantry value is %.4f %s" %(initial_xgantry_value, test_object.xgantry.egu))
logger.info("initial ygantry value is %.4f %s" %(initial_ygantry_value, test_object.ygantry.egu))

2018-07-12 16:16:25,059 - __main__ - INFO - 	1531437384.9805 recording original position...

2018-07-12 16:16:25,064 - __main__ - INFO - initial mirror angle is -53.1453 urad
2018-07-12 16:16:25,066 - __main__ - INFO - initial xgantry value is -4.0036 mm
2018-07-12 16:16:25,068 - __main__ - INFO - initial ygantry value is -7.6010 mm


# X and y checks

In [6]:
verify_axes_coupled(test_object)

2018-07-10 09:30:51,363 - functions - INFO - checking if xrt_m1 axes are coupled
2018-07-10 09:30:51,367 - functions - INFO - xGantry status: coupled
2018-07-10 09:30:51,369 - functions - INFO -  ygantry status: coupled


In [7]:
get_gantry_difference(test_object)

2018-07-10 09:30:54,545 - functions - INFO - getting gantry differences for mirror xrt_m1
2018-07-10 09:30:54,548 - functions - INFO - xgantry difference: -0.0008
2018-07-10 09:30:54,549 - functions - INFO - ygantry_difference: -0.0002


In [8]:
# get 10 second average noise level for x and y position
avg_noise_level(test_object.xgantry)
avg_noise_level(test_object.ygantry)

2018-07-10 09:32:09,802 - functions - INFO - recording 10 second average value ...
2018-07-10 09:32:19,819 - functions - INFO - 10 second noise level (standard deviation) is 0.000014 mm
2018-07-10 09:32:19,822 - functions - INFO - recording 10 second average value ...
2018-07-10 09:32:29,835 - functions - INFO - 10 second noise level (standard deviation) is 0.000011 mm


Request a move, 0.5mm
Record: final settled value of primary axis, final gantry difference, time for move to complete

In [9]:
# running the test with 5 mins timeout
gantry_checks(test_object.xgantry, 0.5, initial_xgantry_value, 300)

2018-07-10 09:32:29,845 - functions - INFO - movement check....
2018-07-10 09:32:29,847 - functions - INFO - moving by 0.500000...


xrt_m1_xgantry:   0%|                                | 0.0/0.5 [00:00<?, ?mm/s]
xrt_m1_xgantry:   0%|                                | 0.0/0.5 [00:00<?, ?mm/s]
xrt_m1_xgantry:   0%|                     | 0.001/0.5 [00:00<03:12, 384.79s/mm]
xrt_m1_xgantry:   0%|                     | 0.002/0.5 [00:00<02:02, 245.57s/mm]
xrt_m1_xgantry:   1%|▏                    | 0.003/0.5 [00:00<01:38, 197.20s/mm]
xrt_m1_xgantry:   1%|▏                    | 0.004/0.5 [00:00<01:25, 172.43s/mm]
xrt_m1_xgantry:   1%|▎                    | 0.006/0.5 [00:00<01:05, 132.51s/mm]
xrt_m1_xgantry:   1%|▎                    | 0.007/0.5 [00:00<01:03, 128.22s/mm]
xrt_m1_xgantry:   2%|▍                    | 0.009/0.5 [00:00<00:54, 110.27s/mm]
xrt_m1_xgantry:   2%|▍                     | 0.011/0.5 [00:01<00:48, 99.24s/mm]
xrt_m1_xgantry:   3%|▌                     | 0.013/0.5 [00:01<00:44, 91.96s/mm]
xrt_m1_xgantry:   3%|▋                     | 0.016/0.5 [00:01<00:39, 80.92s/mm]
xrt_m1_xgantry:   4%|▊                  

2018-07-10 09:32:39,935 - functions - INFO - final gantry value for this axis: -3.5045. Final gantry difference: -0.0019. Time taken for move to complete: 10.1038
2018-07-10 09:32:39,937 - functions - INFO - moving to original position...


In [10]:
gantry_checks(test_object.ygantry, 0.5, initial_ygantry_value, 300)

2018-07-10 09:33:45,031 - functions - INFO - movement check....
2018-07-10 09:33:45,033 - functions - INFO - moving by 0.500000...


xrt_m1_ygantry:   0%|                                | 0.0/0.5 [00:00<?, ?mm/s]
xrt_m1_ygantry:   0%|                                | 0.0/0.5 [00:00<?, ?mm/s]
xrt_m1_ygantry:   0%|                     | 0.001/0.5 [00:00<02:42, 325.07s/mm]
xrt_m1_ygantry:   0%|                     | 0.001/0.5 [00:00<03:37, 435.46s/mm]
xrt_m1_ygantry:   0%|                     | 0.002/0.5 [00:00<02:11, 264.24s/mm]
xrt_m1_ygantry:   1%|▏                    | 0.003/0.5 [00:00<01:44, 210.11s/mm]
xrt_m1_ygantry:   1%|▏                    | 0.005/0.5 [00:00<01:12, 146.52s/mm]
xrt_m1_ygantry:   1%|▎                    | 0.006/0.5 [00:00<01:09, 140.40s/mm]
xrt_m1_ygantry:   2%|▎                    | 0.008/0.5 [00:00<00:57, 117.41s/mm]
xrt_m1_ygantry:   2%|▍                     | 0.01/0.5 [00:01<00:51, 104.38s/mm]
xrt_m1_ygantry:   2%|▌                     | 0.012/0.5 [00:01<00:46, 95.16s/mm]
xrt_m1_ygantry:   3%|▌                     | 0.014/0.5 [00:01<00:43, 88.96s/mm]
xrt_m1_ygantry:   3%|▋                  

2018-07-10 09:33:55,128 - functions - INFO - final gantry value for this axis: -7.0996. Final gantry difference: -0.0003. Time taken for move to complete: 10.1389
2018-07-10 09:33:55,130 - functions - INFO - moving to original position...


# Pitch checks

Record 10 second average noise level

In [11]:
avg_noise_level(test_object.pitch)

2018-07-10 09:34:28,146 - functions - INFO - recording 10 second average value ...
2018-07-10 09:34:38,160 - functions - INFO - 10 second noise level (standard deviation) is 0.055840 urad


Record time for small move (ie. move within piezo range [~20 nrad])

In [12]:
rec_time_for_a_move(test_object, 20/1000, 300)

2018-07-10 09:34:50,124 - functions - INFO - recording time for 0.020000 urad move ... 


xrt_m1_pitch: 0.053urad [00:00,  2.12s/urad]
xrt_m1_pitch:  10%|██                  | 0.002/0.02 [00:00<00:02, 111.61s/urad]
xrt_m1_pitch: 0.08urad [00:00,  4.01s/urad]
xrt_m1_pitch:   0%|                               | 0.0/0.02 [00:00<?, ?urad/s]
xrt_m1_pitch:  10%|██                  | 0.002/0.02 [00:00<00:04, 260.26s/urad]
xrt_m1_pitch: 0.06urad [00:00, 10.40s/urad]
xrt_m1_pitch:  90%|██████████████████▉  | 0.018/0.02 [00:00<00:00, 40.30s/urad]
xrt_m1_pitch: 0.035urad [00:00, 23.71s/urad]
xrt_m1_pitch:  90%|██████████████████▉  | 0.018/0.02 [00:00<00:00, 52.03s/urad]
xrt_m1_pitch: 0.055urad [00:01, 18.76s/urad]
xrt_m1_pitch: 0.023urad [00:01, 49.59s/urad]
xrt_m1_pitch:  95%|███████████████████▉ | 0.019/0.02 [00:01<00:00, 65.04s/urad]
xrt_m1_pitch: 0.055urad [00:01, 24.31s/urad]
xrt_m1_pitch:  90%|██████████████████▉  | 0.018/0.02 [00:01<00:00, 79.91s/urad]
xrt_m1_pitch: 0.035urad [00:01, 44.09s/urad]
xrt_m1_pitch:  20%|████                | 0.004/0.02 [00:01<00:06, 411.48s/urad]
xr

xrt_m1_pitch:  90%|█████████████████  | 0.018/0.02 [00:28<00:03, 1606.40s/urad]
xrt_m1_pitch: 0.098urad [00:29, 296.09s/urad]
xrt_m1_pitch: 0.027urad [00:29, 1078.41s/urad]
xrt_m1_pitch: 0.059urad [00:29, 495.23s/urad]
xrt_m1_pitch: 0.105urad [00:29, 279.29s/urad]
xrt_m1_pitch: 0.119urad [00:29, 247.30s/urad]
xrt_m1_pitch:  50%|██████████          | 0.01/0.02 [00:29<00:29, 2952.48s/urad]
xrt_m1_pitch: 0.076urad [00:29, 389.78s/urad]
xrt_m1_pitch:  95%|██████████████████ | 0.019/0.02 [00:29<00:01, 1564.49s/urad]
xrt_m1_pitch: 0.021urad [00:29, 1420.28s/urad]
xrt_m1_pitch:  20%|███▊               | 0.004/0.02 [00:29<01:59, 7482.23s/urad]
xrt_m1_pitch: 0.066urad [00:30, 454.94s/urad]
xrt_m1_pitch: 0.06urad [00:30, 502.17s/urad]
xrt_m1_pitch: 0.027urad [00:30, 1119.68s/urad]
xrt_m1_pitch: 0.059urad [00:30, 514.14s/urad]
xrt_m1_pitch:  50%|██████████          | 0.01/0.02 [00:30<00:30, 3043.67s/urad]
xrt_m1_pitch: 0.043urad [00:30, 710.20s/urad]
xrt_m1_pitch: 0.027urad [00:30, 1134.97s/urad]

xrt_m1_pitch:  50%|██████████          | 0.01/0.02 [00:57<00:57, 5752.53s/urad]
xrt_m1_pitch:  20%|███▌              | 0.004/0.02 [00:57<03:50, 14408.12s/urad]
xrt_m1_pitch:  70%|█████████████▎     | 0.014/0.02 [00:57<00:24, 4123.43s/urad]
xrt_m1_pitch:  80%|███████████████▏   | 0.016/0.02 [00:57<00:14, 3614.69s/urad]
xrt_m1_pitch: 0.06urad [00:57, 965.59s/urad]
xrt_m1_pitch: 0.066urad [00:58, 879.41s/urad]
xrt_m1_pitch:  60%|███████████▍       | 0.012/0.02 [00:58<00:38, 4845.10s/urad]
xrt_m1_pitch:  10%|█▊                | 0.002/0.02 [00:58<08:44, 29118.99s/urad]
xrt_m1_pitch:  90%|█████████████████  | 0.018/0.02 [00:58<00:06, 3241.11s/urad]
xrt_m1_pitch:  60%|███████████▍       | 0.012/0.02 [00:58<00:38, 4870.39s/urad]
xrt_m1_pitch: 0.064urad [00:58, 914.84s/urad]
xrt_m1_pitch:  70%|█████████████▎     | 0.014/0.02 [00:58<00:25, 4188.97s/urad]
xrt_m1_pitch: 0.094urad [00:58, 624.98s/urad]
xrt_m1_pitch: 0.051urad [00:58, 1153.91s/urad]
xrt_m1_pitch: 0.029urad [00:58, 2032.85s/urad]
xrt

TypeError: info() missing 1 required positional argument: 'msg'

Record time for large move (ie. move > 60 nrad)

In [13]:
rec_time_for_a_move(test_object, 100/1000, 300)

2018-07-10 09:36:26,607 - functions - INFO - recording time for 0.100000 urad move ... 


xrt_m1_pitch:  80%|██████████████████▍    | 0.08/0.1 [00:00<00:00,  2.64s/urad]
xrt_m1_pitch:  29%|██████▍               | 0.029/0.1 [00:00<00:00, 10.94s/urad]
xrt_m1_pitch:  10%|██▎                    | 0.01/0.1 [00:00<00:03, 41.26s/urad]
xrt_m1_pitch:  43%|█████████▍            | 0.043/0.1 [00:00<00:00, 12.01s/urad]
xrt_m1_pitch:  18%|███▉                  | 0.018/0.1 [00:00<00:02, 34.80s/urad]
xrt_m1_pitch:  43%|█████████▍            | 0.043/0.1 [00:00<00:00, 16.82s/urad]
xrt_m1_pitch:  33%|███████▎              | 0.033/0.1 [00:00<00:01, 24.92s/urad]
xrt_m1_pitch:  60%|█████████████▊         | 0.06/0.1 [00:00<00:00, 15.33s/urad]
xrt_m1_pitch:  31%|██████▊               | 0.031/0.1 [00:01<00:02, 33.08s/urad]
xrt_m1_pitch:  60%|█████████████▊         | 0.06/0.1 [00:01<00:00, 18.73s/urad]
xrt_m1_pitch:  31%|██████▊               | 0.031/0.1 [00:01<00:02, 39.55s/urad]
xrt_m1_pitch:  20%|████▌                  | 0.02/0.1 [00:01<00:05, 66.31s/urad]
xrt_m1_pitch:  33%|███████▎             

xrt_m1_pitch:  16%|███▏                | 0.016/0.1 [00:21<01:54, 1358.19s/urad]
xrt_m1_pitch: 0.119urad [00:21, 183.45s/urad]
xrt_m1_pitch: 0.101urad [00:21, 217.20s/urad]
xrt_m1_pitch:  29%|██████               | 0.029/0.1 [00:22<00:53, 759.84s/urad]
xrt_m1_pitch: 0.109urad [00:22, 203.06s/urad]
xrt_m1_pitch: 0.144urad [00:22, 154.43s/urad]
xrt_m1_pitch:  94%|███████████████████▋ | 0.094/0.1 [00:22<00:01, 237.61s/urad]
xrt_m1_pitch:  27%|█████▋               | 0.027/0.1 [00:22<01:00, 830.97s/urad]
xrt_m1_pitch:  82%|█████████████████▏   | 0.082/0.1 [00:22<00:04, 274.86s/urad]
xrt_m1_pitch: 0.177urad [00:22, 127.91s/urad]
xrt_m1_pitch: 0.148urad [00:22, 153.65s/urad]
xrt_m1_pitch:  88%|██████████████████▍  | 0.088/0.1 [00:22<00:03, 259.56s/urad]
xrt_m1_pitch:  99%|████████████████████▊| 0.099/0.1 [00:22<00:00, 231.76s/urad]
xrt_m1_pitch:  62%|█████████████        | 0.062/0.1 [00:23<00:14, 371.69s/urad]
xrt_m1_pitch: 0.154urad [00:23, 150.30s/urad]
xrt_m1_pitch: 0.172urad [00:23, 135.15

xrt_m1_pitch:  45%|█████████           | 0.045/0.1 [00:47<00:58, 1059.68s/urad]
xrt_m1_pitch: 0.101urad [00:47, 473.07s/urad]
xrt_m1_pitch:  82%|█████████████████▏   | 0.082/0.1 [00:47<00:10, 583.93s/urad]
xrt_m1_pitch:  99%|████████████████████▊| 0.099/0.1 [00:47<00:00, 484.68s/urad]
xrt_m1_pitch:  43%|████████▌           | 0.043/0.1 [00:48<01:03, 1118.27s/urad]
xrt_m1_pitch:  82%|█████████████████▏   | 0.082/0.1 [00:48<00:10, 587.65s/urad]
xrt_m1_pitch:  64%|█████████████▍       | 0.064/0.1 [00:48<00:27, 754.53s/urad]
xrt_m1_pitch:  84%|█████████████████▋   | 0.084/0.1 [00:48<00:09, 576.14s/urad]
xrt_m1_pitch:  66%|█████████████▊       | 0.066/0.1 [00:48<00:24, 734.75s/urad]
xrt_m1_pitch: 0.162urad [00:48, 299.98s/urad]
xrt_m1_pitch: 0.148urad [00:48, 329.07s/urad]
xrt_m1_pitch: 0.113urad [00:48, 431.89s/urad]
xrt_m1_pitch:  99%|████████████████████▊| 0.099/0.1 [00:48<00:00, 493.96s/urad]
xrt_m1_pitch: 0.115urad [00:49, 426.16s/urad]
xrt_m1_pitch: 0.127urad [00:49, 386.72s/urad]
xrt_

xrt_m1_pitch: 0.146urad [01:13, 504.87s/urad]
xrt_m1_pitch:  97%|████████████████████▎| 0.097/0.1 [01:13<00:02, 760.97s/urad]
xrt_m1_pitch: 0.133urad [01:13, 555.69s/urad]
xrt_m1_pitch:  94%|███████████████████▋ | 0.094/0.1 [01:14<00:04, 787.31s/urad]
xrt_m1_pitch:  43%|████████▌           | 0.043/0.1 [01:14<01:38, 1723.49s/urad]
xrt_m1_pitch: 0.183urad [01:14, 405.53s/urad]
xrt_m1_pitch:  72%|██████████████▍     | 0.072/0.1 [01:14<00:28, 1032.16s/urad]
xrt_m1_pitch: 0.105urad [01:14, 708.73s/urad]
xrt_m1_pitch:  57%|███████████▍        | 0.057/0.1 [01:14<00:56, 1307.33s/urad]
xrt_m1_pitch:  21%|████▏               | 0.021/0.1 [01:14<04:40, 3553.40s/urad]
xrt_m1_pitch: 0.199urad [01:14, 375.48s/urad]
xrt_m1_pitch: 0.117urad [01:14, 639.58s/urad]
xrt_m1_pitch: 0.142urad [01:14, 527.66s/urad]
xrt_m1_pitch:  33%|██████▌             | 0.033/0.1 [01:15<02:32, 2273.87s/urad]
xrt_m1_pitch: 0.154urad [01:15, 487.86s/urad]
xrt_m1_pitch:  70%|██████████████▋      | 0.07/0.1 [01:15<00:32, 1074.77

...

xrt_m1_pitch: 0.115urad [03:28, 1812.05s/urad]
xrt_m1_pitch: 0.133urad [03:28, 1567.60s/urad]
xrt_m1_pitch:  99%|███████████████████▊| 0.099/0.1 [03:28<00:02, 2106.98s/urad]
xrt_m1_pitch:   6%|█▏                 | 0.006/0.1 [03:28<54:29, 34781.97s/urad]
xrt_m1_pitch:  72%|██████████████▍     | 0.072/0.1 [03:28<01:21, 2900.01s/urad]
xrt_m1_pitch:  45%|█████████           | 0.045/0.1 [03:28<04:15, 4642.33s/urad]
xrt_m1_pitch: 0.156urad [03:29, 1339.76s/urad]
xrt_m1_pitch:  99%|███████████████████▊| 0.099/0.1 [03:29<00:02, 2112.11s/urad]
xrt_m1_pitch:  86%|█████████████████▏  | 0.086/0.1 [03:29<00:34, 2432.70s/urad]
xrt_m1_pitch:  64%|████████████▊       | 0.064/0.1 [03:29<01:57, 3272.01s/urad]
xrt_m1_pitch:  80%|████████████████▊    | 0.08/0.1 [03:29<00:52, 2618.92s/urad]
xrt_m1_pitch:  70%|██████████████▋      | 0.07/0.1 [03:29<01:29, 2994.53s/urad]
xrt_m1_pitch:  84%|████████████████▊   | 0.084/0.1 [03:29<00:39, 2496.59s/urad]
xrt_m1_pitch:  96%|███████████████████▏| 0.096/0.1 [03:29<0

xrt_m1_pitch: 0.14urad [03:55, 1678.71s/urad]
xrt_m1_pitch: 0.142urad [03:55, 1655.79s/urad]
xrt_m1_pitch: 0.119urad [03:55, 1976.68s/urad]
xrt_m1_pitch:  66%|█████████████▏      | 0.066/0.1 [03:55<02:01, 3565.46s/urad]
xrt_m1_pitch: 0.187urad [03:55, 1258.94s/urad]
xrt_m1_pitch: 0.105urad [03:55, 2243.12s/urad]
xrt_m1_pitch:  86%|█████████████████▏  | 0.086/0.1 [03:55<00:38, 2739.90s/urad]
xrt_m1_pitch: 0.121urad [03:55, 1948.17s/urad]
xrt_m1_pitch: 0.109urad [03:55, 2163.60s/urad]
xrt_m1_pitch: 0.107urad [03:55, 2204.95s/urad]
xrt_m1_pitch:  12%|██▎                | 0.012/0.1 [03:56<28:50, 19669.96s/urad]
xrt_m1_pitch: 0.113urad [03:56, 2089.68s/urad]
xrt_m1_pitch: 0.164urad [03:56, 1440.45s/urad]
xrt_m1_pitch:  80%|████████████████▊    | 0.08/0.1 [03:56<00:59, 2954.20s/urad]
xrt_m1_pitch: 0.125urad [03:56, 1891.50s/urad]
xrt_m1_pitch: 0.115urad [03:56, 2056.84s/urad]
xrt_m1_pitch: 0.189urad [03:56, 1252.05s/urad]
xrt_m1_pitch:  27%|█████▍              | 0.027/0.1 [03:56<10:40, 8768.

xrt_m1_pitch: 0.174urad [04:21, 1504.56s/urad]
xrt_m1_pitch: 0.15urad [04:21, 1745.97s/urad]
xrt_m1_pitch: 0.111urad [04:21, 2360.31s/urad]
xrt_m1_pitch: 0.105urad [04:22, 2496.16s/urad]
xrt_m1_pitch:   8%|█▌                 | 0.008/0.1 [04:22<50:15, 32775.00s/urad]
xrt_m1_pitch:  62%|████████████▍       | 0.062/0.1 [04:22<02:40, 4230.64s/urad]
xrt_m1_pitch:  12%|██▎                | 0.012/0.1 [04:22<32:04, 21866.74s/urad]
xrt_m1_pitch: 0.105urad [04:22, 2500.00s/urad]
xrt_m1_pitch:  31%|██████▏             | 0.031/0.1 [04:22<09:44, 8470.99s/urad]
xrt_m1_pitch: 0.123urad [04:22, 2135.79s/urad]
xrt_m1_pitch: 0.152urad [04:22, 1729.00s/urad]
xrt_m1_pitch:  97%|███████████████████▍| 0.097/0.1 [04:22<00:08, 2710.36s/urad]
xrt_m1_pitch: 0.232urad [04:23, 1133.67s/urad]
xrt_m1_pitch: 0.152urad [04:23, 1730.97s/urad]
xrt_m1_pitch:  96%|███████████████████▏| 0.096/0.1 [04:23<00:10, 2741.81s/urad]
xrt_m1_pitch: 0.166urad [04:23, 1586.22s/urad]
xrt_m1_pitch:  49%|█████████▊          | 0.049/0.1 

xrt_m1_pitch:  84%|████████████████▊   | 0.084/0.1 [04:48<00:54, 3430.80s/urad]
xrt_m1_pitch:  55%|██████████▉         | 0.055/0.1 [04:48<03:55, 5241.68s/urad]
xrt_m1_pitch:  97%|███████████████████▍| 0.097/0.1 [04:48<00:08, 2973.08s/urad]
xrt_m1_pitch: 0.125urad [04:48, 2307.96s/urad]
xrt_m1_pitch:  74%|██████████████▊     | 0.074/0.1 [04:48<01:41, 3899.92s/urad]
xrt_m1_pitch:  92%|██████████████████▍ | 0.092/0.1 [04:48<00:25, 3138.00s/urad]
xrt_m1_pitch:  97%|███████████████████▍| 0.097/0.1 [04:48<00:08, 2977.28s/urad]
xrt_m1_pitch:  74%|██████████████▊     | 0.074/0.1 [04:48<01:41, 3904.04s/urad]
xrt_m1_pitch:  84%|████████████████▊   | 0.084/0.1 [04:49<00:55, 3440.51s/urad]
xrt_m1_pitch:  94%|██████████████████▊ | 0.094/0.1 [04:49<00:18, 3075.60s/urad]
xrt_m1_pitch:  92%|██████████████████▍ | 0.092/0.1 [04:49<00:25, 3143.56s/urad]
xrt_m1_pitch: 0.135urad [04:49, 2143.02s/urad]
xrt_m1_pitch:  60%|████████████▌        | 0.06/0.1 [04:49<03:12, 4823.58s/urad]
xrt_m1_pitch: 0.105urad [0

RuntimeError: Operation completed but reported an error

# for all axes

Request a large move, change request before move is complete (small delay), verify target change is accepted and final position matches second command


xgantry

In [14]:
low_lim, hi_lim = low_high_limit(test_object.xgantry, 600)
request_changeRequest(test_object.xgantry, initial_xgantry_value, hi_lim - 1, hi_lim - 0.4, 300)

2018-07-10 09:42:45,806 - functions - INFO - going to low limit...
2018-07-10 09:43:15,715 - functions - WARNING - you hit low soft limit switch at -6.735287189483643
2018-07-10 09:43:15,717 - functions - INFO - going to high limit...
2018-07-10 09:48:19,683 - functions - WARNING - you hit high soft limit switch at 0.8111069798469543


KeyboardInterrupt: 

ygantry

In [ ]:
low_lim, hi_lim = low_high_limit(test_object.ygantry, 600)
request_changeRequest(test_object.ygantry, initial_ygantry_value, hi_lim - 1, hi_lim - 0.4, 300)

pitch

In [6]:
low_lim, hi_lim = low_high_limit(test_object.pitch.position, 600)
request_changeRequest(test_object.pitch, initial_mirror_angle, hi_lim -1, hi_lim - 0.4 , 300)

2018-07-12 16:17:06,832 - functions - INFO - going to low limit...
2018-07-12 16:17:06,835 - functions - INFO - going to high limit...


AttributeError: 'float' object has no attribute 'done'

In [15]:
# Restore all axes to original position
logger.info("restoring x value to original position...")
test_object.xgantry.move(initial_xgantry_value)
logger.info("restoring y value to original position...")
test_object.ygantry.move(initial_ygantry_value)
logger.info("restoring pitch to original position...")
test_object.pitch.move(initial_mirror_angle)
logger.info("done")

2018-07-10 09:52:46,907 - __main__ - INFO - restoring x value to original position...
2018-07-10 09:52:47,368 - __main__ - INFO - restoring y value to original position...
2018-07-10 09:53:11,727 - __main__ - INFO - restoring pitch to original position...


NameError: name 'initial_pitch_value' is not defined